<a href="https://colab.research.google.com/github/sadhna7/Chatbot/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Unzip the json file
from zipfile import ZipFile
with ZipFile('/content/json-file.zip', 'r') as zip:
  zip.extractall()

In [3]:
# importing necessary libraries 
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer() # for getting the root of words,stemmer is used

import numpy
import tflearn
import tensorflow
import random

import json
import pickle
with open("/content/json file/intent.json") as file:
  data = json.load(file)

try:
  with open("data.pickle","rb") as f:
    words,labels,training,output = pickle.load(f)
except:
  words =[]
  labels =[]
  docs_x =[]
  docs_y =[]
# here, we are looping through the intents:
  for intent in data["intents"]:
    for pattern in intent["patterns"]:
      wrds = nltk.word_tokenize(pattern) # tokenizing the words in pattern
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(intent["tag"])

      if intent["tag"] not in labels:
        labels.append(intent["tag"])

  words = [stemmer.stem(w.lower()) for w in words if w != "?"]
  words = sorted(list(set(words))) # avoiding duplicate words so as
                                   # to make our word vocab shorter

  labels = sorted(labels)

  # now, we are going to train it, but the neural network doesnot accepts 
  # strings as input
  # so we will create a bag of words to convert strings into numeric data.

  training=[]
  output =[]

  out_empty = [ 0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):
    bag=[]
    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words: 
      if w in wrds:
        bag.append(1)  # if the word exist in doc, we will give it 1 otherwise 0
      else:
        bag.append(0)
  
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

  # converting it into numpy arrays so to feed them into our model 
  training = numpy.array(training)                                   
  output = numpy.array(output)

  with open("data.pickle","wb") as f:
    pickle.dump((words,labels,training,output), f)

# building our model
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None,len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]),activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training,output,n_epoch =1000,batch_size = 8 , show_metric = True)
model.save("model.tflearn")

try:
  model.load("model.tflearn")
except:
  model.fit(training,output,n_epoch = 1000,batch_size=8, show_metric = True)
  model.save("model.tflearn")

Training Step: 3999  | total loss: 0.22044 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.22044 - acc: 0.9476 -- iter: 24/26
Training Step: 4000  | total loss: 0.21129 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.21129 - acc: 0.9529 -- iter: 26/26
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [5]:
# making predictions
def bag_of_words(s,words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i,w in enumerate(words):
      if w == se:
        bag[i]=1

  return numpy.array(bag)

def chat():
  print("start talking with the bot(type quit to stop)!")
  while True:
    inp = input("You: ")
    if inp.lower()=="quit":
      break
    results = model.predict([bag_of_words(inp,words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    if results[results_index]>0.7:
      for tg in data["intents"]:
        if tg["tag"]==tag:
          responses = tg["responses"]
      print(random.choice(responses))
    else:
      print("I didn't get that,try again")

chat()

start talking with the bot(type quit to stop)!
You: hello
Good to see you again!
You: from where are you
My hometown is Ambala cantt
You: what is your name
You can call me Sadhna.
You: what do you like
I didn't get that,try again
You: what do you like to have
Yes, I would like to have Coffee
You: quit
